In [1]:
import pandas as pd
from casauth import CASAuth
import swat
import os

path = os.getenv('CAS_CREDENTIALS')
pem_file = os.getenv('CAS_CLIENT_SSL_CA_LIST')

conn = CASAuth(path, ssl_ca_list = pem_file)
conn.about()['About']['Viya Version']

CAS Connection created
NOTE: Grid node action status report: 5 nodes, 9 total actions executed.


'Stable 2023.05'

## Create data

### Create DataFrame

In [74]:
df = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df

,A,B
0,1,4
1,2,5
2,3,6


### Create CAS table

In [76]:
castbl = conn.upload_frame(df, casout='CAS_TBL')
castbl.head()

NOTE: Cloud Analytic Services made the uploaded file available as table CAS_TBL in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table CAS_TBL has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


,A,B
0,1.0,4.0
1,2.0,5.0
2,3.0,6.0


## Rename columns using a dict

### Pandas You need true to rename the columns permanently

In [77]:
df.rename(columns = {'A':'NEWA', 'B':'NEWB'}, inplace = True)

In [78]:
df.head()

,NEWA,NEWB
0,1,4
1,2,5
2,3,6


### Cas table uses alterTable action. Requires list of dicts

In [79]:
castbl.alterTable(columns = [{'name':'A', 'rename':'NEWA'}, {'name':'B','rename':'NEWB'}])

+ Elapsed: 0.00985s, user: 0.00486s, sys: 0.0156s, mem: 1.53mb

In [80]:
castbl.head()

,NEWA,NEWB
0,1.0,4.0
1,2.0,5.0
2,3.0,6.0


### So for CAS you would most likely need a list comprehension

In [81]:
## store the dict from columns
columns = {'NEWA':'BACK2A', 'NEWB':'BACK2B'}

## Create list of dictionaries
myListOfDicts = [{'name':key, 'rename':value} for key,value in columns.items()]
myListOfDicts

[{'name': 'NEWA', 'rename': 'BACK2A'}, {'name': 'NEWB', 'rename': 'BACK2B'}]

In [82]:
## Apply change
castbl.alterTable(columns = myListOfDicts)

+ Elapsed: 0.0103s, user: 0.0122s, sys: 0.00993s, mem: 1.52mb

In [83]:
castbl.head()

,BACK2A,BACK2B
0,1.0,4.0
1,2.0,5.0
2,3.0,6.0


## Rename columns using a function

###  Pandas 
Create a python function

In [63]:
df.head()

,NEWA,NEWB
0,1,4
1,2,5
2,3,6


In [64]:
def clean_col(_col):
    return _col.strip().lower().replace(' ','_')

df.rename(columns=clean_col, inplace = True)
df.head()

,newa,newb
0,1,4
1,2,5
2,3,6


Using a lambda function

In [65]:
df.rename(columns = lambda col: col.upper(), inplace = True)
df.head()

,NEWA,NEWB
0,1,4
1,2,5
2,3,6


### CAS dynamic rename using function
You would have to do something like this in CAS

In [84]:
castbl.head()

,BACK2A,BACK2B
0,1.0,4.0
1,2.0,5.0
2,3.0,6.0


In [85]:
# Store CAS table columns names in a list
columnNames = castbl.columns.to_list()
columnNames

['BACK2A', 'BACK2B']

In [86]:
myListOfDicts = [{'name':col, 'rename':col.lower()} for col in columnNames]
myListOfDicts

[{'name': 'BACK2A', 'rename': 'back2a'},
 {'name': 'BACK2B', 'rename': 'back2b'}]

In [87]:
castbl.alterTable(columns = myListOfDicts)

+ Elapsed: 0.00923s, user: 0.00701s, sys: 0.013s, mem: 1.52mb

In [88]:
castbl.head()

,back2a,back2b
0,1.0,4.0
1,2.0,5.0
2,3.0,6.0
